In [1]:
# Based on  tutorial https://github.com/h2oai/h2o-tutorials/blob/master/h2o-world-2017/automl/Python/automl_regression_powerplant_output.ipynb
# Try to rerun model 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
#h2o.shutdown(prompt=True) 

In [5]:
# Needed to install Java JRE https://www.java.com/en/download/win10.jsp 
import h2o 
from h2o.automl import H2OAutoML
h2o.init(ip="localhost", port=54323)

Checking whether there is an H2O instance running at http://localhost:54323 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.221-b11, mixed mode, sharing)


C:\Users\dmmma\Anaconda3\lib\site-packages\h2o\backend\server.py:369: UserWarning:   You have a 32-bit version of Java. H2O works best with 64-bit Java.
  Please download the latest 64-bit Java SE JDK from Oracle.

  warn("  You have a 32-bit version of Java. H2O works best with 64-bit Java.\n"


  Starting server from C:\Users\dmmma\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\dmmma\AppData\Local\Temp\tmp7xivitfg
  JVM stdout: C:\Users\dmmma\AppData\Local\Temp\tmp7xivitfg\h2o_dmmma_started_from_python.out
  JVM stderr: C:\Users\dmmma\AppData\Local\Temp\tmp7xivitfg\h2o_dmmma_started_from_python.err
  Server is running at http://127.0.0.1:54323
Connecting to H2O server at http://127.0.0.1:54323 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.2
H2O cluster version age:,19 days
H2O cluster name:,H2O_from_python_dmmma_2qso65
H2O cluster total nodes:,1
H2O cluster free memory:,247.5 Mb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


### Load data

In [6]:
# Setting 'parse_dates' in this case parses both dates and times
# These files are too large to commit so they're uploaded locally under `/exploring_models` but not pushed
train_raw = pd.read_csv('weather1_education_train.csv', parse_dates = ['timestamp'])
val_raw = pd.read_csv('weather1_education_test.csv', parse_dates = ['timestamp'])

In [7]:
train_data = train_raw.drop([ 'building_name', 'primary_space_usage', 'hour', 'year', 'month','weekday', 'date', 'timestamp', 'Unnamed: 0'], axis=1)
val_data = val_raw.drop([ 'building_name', 'primary_space_usage', 'hour', 'year', 'month','weekday', 'date', 'timestamp', 'Unnamed: 0'], axis=1)

train_data.head()


,area,electricity,TemperatureC,month_1,month_2,month_3,month_4,month_5,month_6,month_7,...,hour_23,wkday_0,wkday_1,wkday_2,wkday_3,wkday_4,wkday_5,wkday_6,PSU_PrimClass,PSU_UnivClass
0,2777.0,5.0,7.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,2777.0,5.2,5.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,2777.0,5.2,5.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,2777.0,5.5,6.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,2777.0,6.0,7.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [8]:
val_data.head()

,area,electricity,TemperatureC,month_1,month_2,month_3,month_4,month_5,month_6,month_7,...,hour_23,wkday_0,wkday_1,wkday_2,wkday_3,wkday_4,wkday_5,wkday_6,PSU_PrimClass,PSU_UnivClass
0,6149.0,34.900002,7.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,6149.0,32.700000,5.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,6149.0,32.200000,5.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,6149.0,32.300000,6.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,6149.0,32.599998,7.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


### Fit a model

In [9]:
# Using one train/val split, not walk-forward cross-val
# In h2o, if using all other cols for x, don't have to explicitly specify
# Takes > 1 min to run

# Must be formatted as h2o frames http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/frame.html
train =  h2o.H2OFrame(train_data)
test = h2o.H2OFrame(val_data)


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [10]:
train.columns[:10]

['area',
 'electricity',
 'TemperatureC',
 'month_1',
 'month_2',
 'month_3',
 'month_4',
 'month_5',
 'month_6',
 'month_7']

In [11]:
# In past, running for 60 s only fit one model (GLM), 3 min fit 3 (GLM, GBM, and Stacked Ensemble)
y = 'electricity'
x = list(train.columns)
x.remove(y)


In [12]:
aml =  H2OAutoML(max_runtime_secs = 60*15, seed = 1)
aml.train(x = x, y = y, training_frame = train) 
# https://h2o-release.s3.amazonaws.com/h2o/rel-xia/1/docs-website/h2o-docs/automl.html
# "If the user doesn’t specify a validation_frame, then one will be created automatically by randomly partitioning the training data"

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [13]:
aml.leaderboard.head()

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GLM_grid_1_AutoML_20190815_205904_model_1,355.912,18.8656,355.912,11.3939,nan


In [14]:
# Inspect the leader model
aml.leader

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_grid_1_AutoML_20190815_205904_model_1

GLM Model: summary



,family,link,regularization,lambda_search,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
,gaussian,identity,Ridge ( lambda = 0.003383 ),"nlambda = 30, lambda.max = 2100.8, lambda.min = 0.003383, lambda.1se = 0.02274",78,78,30,automl_training_Key_Frame__upload_bc82be3fe1bb75f76654e57af7a4489a.hex




ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 355.90198634165347
RMSE: 18.86536472856153
MAE: 11.398359620762543
RMSLE: NaN
R^2: 0.610634857329595
Mean Residual Deviance: 355.90198634165347
Null degrees of freedom: 437849
Residual degrees of freedom: 437771
Null deviance: 400219916.0688677
Residual deviance: 155831684.71969298
AIC: 3814942.328892991

ModelMetricsRegressionGLM: glm
** Reported on cross-validation data. **

MSE: 355.9119715233938
RMSE: 18.86562936992545
MAE: 11.39389943154019
RMSLE: NaN
R^2: 0.6106239333059547
Mean Residual Deviance: 355.9119715233938
Null degrees of freedom: 437849
Residual degrees of freedom: 437771
Null deviance: 400219920.4307822
Residual deviance: 155836056.73151797
AIC: 3814954.613034871
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,12.9113245,2.0827684,11.397179,11.53778,18.801388,11.413568,11.40671
mean_residual_deviance,489.55487,153.12405,354.73022,465.2227,914.32043,357.08102,356.41998
mse,489.55487,153.12405,354.73022,465.2227,914.32043,357.08102,356.41998
null_deviance,80043984.0000000,89050.21,79874016.0000000,79916616.0000000,80106560.0000000,8.01982E7,80124528.0000000
r2,0.4644730,0.1673187,0.6110910,0.4902242,0.0004933,0.6100962,0.6104601
residual_deviance,42870320.0000000,13409073.0000000,31063726.0000000,40739548.0000000,80067040.0000000,31269584.0000000,31211698.0000000
rmse,21.683342,3.1134808,18.834284,21.569021,30.237732,18.896587,18.87909
rmsle,1.2745814,0.0,NaN,NaN,1.2745814,NaN,NaN


Scoring History: 


,timestamp,duration,iteration,lambda,predictors,deviance_train,deviance_test,deviance_xval,deviance_se
,2019-08-15 21:00:58,0.000 sec,1,.21E4,76,913.4931216,nan,913.6058648,0.7189761
,2019-08-15 21:00:58,0.031 sec,2,.13E4,77,913.1494122,nan,913.3307772,0.7189256
,2019-08-15 21:00:58,0.062 sec,3,.81E3,77,912.5966543,nan,912.8882653,0.7188445
,2019-08-15 21:00:58,0.109 sec,4,.5E3,78,911.7084617,nan,912.1769190,0.7187138
,2019-08-15 21:00:58,0.140 sec,5,.31E3,79,910.2832189,nan,911.0345970,0.7185036
---,---,---,---,---,---,---,---,---,---
,2019-08-15 21:00:59,0.812 sec,26,.14E-1,79,356.0097341,nan,356.0789325,0.4500089
,2019-08-15 21:00:59,0.843 sec,27,.88E-2,79,355.9399134,nan,355.9708828,0.4478193
,2019-08-15 21:00:59,0.875 sec,28,.54E-2,79,355.9126065,nan,355.9284929,0.4464418
,2019-08-15 21:00:59,0.906 sec,29,.34E-2,79,355.9019864,nan,355.9119716,0.4455801



See the whole table with table.as_data_frame()


In [15]:
# make predictions using leader model
pred = aml.leader.predict(test)
pred.head()

glm prediction progress: |████████████████████████████████████████████████| 100%


predict
27.371
27.3404
27.3579
27.4195
27.9138
28.9514
31.8214
36.8104
42.6961
47.0709


In [16]:
pred.shape

(166383, 1)

In [17]:
num_buildings = len(val_raw['building_name'].unique())
print(num_buildings)

19


In [18]:
aml_pred_npy = pred.as_data_frame().as_matrix().reshape((num_buildings, pred.shape[0]//num_buildings))


C:\Users\dmmma\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [19]:
aml_pred_npy.shape

(19, 8757)